In [1]:
# Various torch packages
import torch
import torch.nn as nn
import torch.nn.functional as F

# torchvision
from torchvision import datasets, transforms

# ------------------------
# get up one directory 
import sys, os
sys.path.append(os.path.abspath('../'))
# ------------------------

# custom packages
import models.aux_funs as maf
import optimizers as op
import regularizers as reg
import train
import math
import utils.configuration as cf
import utils.datasets as ud
from utils.datasets import get_data_set, GaussianSmoothing
from models.fully_connected import fully_connected
from models.resnet import ResNet18

In [2]:
# -----------------------------------------------------------------------------------
# Fix random seed
# -----------------------------------------------------------------------------------
random_seed = 2
cf.seed_torch(random_seed)

In [3]:
# -----------------------------------------------------------------------------------
# Parameters
# -----------------------------------------------------------------------------------
conf_args = {#
    # data specification
    'data_file':"../../Data", 'train_split' : 0.95, 'data_set':"CIFAR10",
    # cuda
    'use_cuda': True, 'num_workers':4, 'cuda_device':0, 'pin_memory':True,
    #
    'epochs':100,
    # optimizer
    'delta': 1.0, 'lr': 0.001, 'mu_1': 0.1, 'mu_2':0.0, 'optim':"AdaBreg", 'conv_group':True,
    'beta':0.0,
    # initialization
    'sparse_init':0.01, 'r':[1,5,1],
    # misc
    'random_seed':random_seed, 'eval_acc':True,
}
conf = cf.Conf(**conf_args)

In [4]:
# -----------------------------------------------------------------------------------
# define the model and an instance of the best model class
# -----------------------------------------------------------------------------------
model_kwargs = {'mean':conf.data_set_mean, 'std':conf.data_set_std, 'im_size':conf.im_shape[1],
                'im_channels':conf.im_shape[0]}    

model = ResNet18(mean = conf.data_set_mean, std = conf.data_set_std)
best_model = train.best_model(ResNet18(mean = conf.data_set_mean, std = conf.data_set_std).to(conf.device))
    
# sparsify
maf.sparse_bias_uniform_(model, 0,conf.r[0])
maf.sparse_weight_normal_(model, conf.r[1])
maf.sparsify_(model, conf.sparse_init, conv_group = conf.conv_group)
model = model.to(conf.device)

In [5]:
# -----------------------------------------------------------------------------------
# define the model and an instance of the best model class
# -----------------------------------------------------------------------------------
model_kwargs = {'mean':conf.data_set_mean, 'std':conf.data_set_std}    

def init_weights(conf, model):
    # sparsify
    maf.sparse_bias_uniform_(model, 0, conf.r[0])
    maf.sparse_weight_normal_(model, conf.r[1])
    maf.sparse_weight_normal_(model, conf.r[1],ltype=nn.Conv2d)
    maf.sparsify_(model, conf.sparse_init, ltype = nn.Conv2d, conv_group = conf.conv_group)
    model = model.to(conf.device)
    
    return model

# Optimizer

In [6]:
# -----------------------------------------------------------------------------------
# Optimizer
# -----------------------------------------------------------------------------------
def init_opt(conf, model):
    # Get access to different model parameters
    weights_conv = maf.get_weights_conv(model)
    weights_linear = maf.get_weights_linear(model)
    weights_batch = maf.get_weights_batch(model)
    biases = maf.get_bias(model)

   
    if conf.conv_group:
        reg_1 = reg.reg_l1_l2_conv(conf.mu_1)
    else:
        reg_1 = reg.reg_l1(conf.mu)
        
    reg_2 = reg.reg_l1(conf.mu_2)
        
        
    if conf.optim == "SGD":
        opt = torch.optim.SGD(model.parameters(), lr=lr, momentum=beta)
    elif conf.optim == "LinBreg":
        opt = op.LinBreg([{'params': weights_conv, 'lr' : conf.lr, 'reg' : reg_1, 'momentum':conf.beta},
                          {'params': weights_linear, 'lr' : conf.lr, 'reg' : reg_2, 'momentum':conf.beta},
                          {'params': weights_batch, 'lr' : conf.lr, 'momentum':beta},
                          {'params': biases, 'lr': conf.lr, 'momentum':beta}])
    elif conf.optim == "AdaBreg":
        opt = op.AdaBreg([{'params': weights_conv, 'lr' : conf.lr, 'reg' : reg_1},
                           {'params': weights_linear, 'lr' : conf.lr, 'reg' : reg_2},
                           {'params': weights_batch, 'lr' : conf.lr, 'momentum':conf.beta},
                           {'params': biases, 'lr': conf.lr}])
    elif conf.optim == "L1SGD":
        def weight_reg(model):
            reg_1 =  reg.reg_l1(mu=conf.mu_1)
            
            loss2 = 0
            for w in maf.get_weights_conv(model):
                loss2 += reg2(w)
                
            loss1 = 0
            for w in maf.get_weights_linear(model):
                loss1 += reg_1(w)
                
            return loss1 + loss2
        
        conf.weight_reg = weight_reg
        
        opt = torch.optim.SGD(model.parameters(), lr=conf.lr, momentum=conf.beta)
    else:
        raise ValueError("Unknown Optimizer specified")

    # learning rate scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, factor=0.5, patience=5,threshold=0.01)
    
    return opt, scheduler

# Dataset

In [7]:
train_loader, valid_loader, test_loader = ud.get_data_set(conf)

# History and Runs

In [8]:
# Initialize history
tracked = ['loss', 'node_sparse']
train_history = {key: [] for key in tracked}
val_history = {key: [] for key in tracked}

# Initialize runs
runs = cf.run(**{'num_runs':0})

# Training

In [ ]:
while runs.step():
    # -----------------------------------------------------------------------------------
    # Reinit weigts and the corresponding optimizer
    # -----------------------------------------------------------------------------------
    model = init_weights(conf, model)
    opt, scheduler = init_opt(conf, model)
    
    # -----------------------------------------------------------------------------------
    # train the model
    # -----------------------------------------------------------------------------------
    for epoch in range(conf.epochs):
        print(25*"<>")
        print(50*"|")
        print(25*"<>")
        print('Epoch:', epoch)

        # ------------------------------------------------------------------------
        # train step, log the accuracy and loss
        # ------------------------------------------------------------------------
        train_data = train.train_step(conf, model, opt, train_loader)

        # update history
        for key in tracked:
            if key in train_data:
                train_history[key].append(train_data[key])        

        # ------------------------------------------------------------------------
        # validation step
        val_data = train.validation_step(conf, model, opt, valid_loader, opt_reg_eval=False)

        # update history
        for key in tracked:
            if key in val_data:
                val_history[key].append(val_data[key])

        for i,reg_val in enumerate(val_data['node_sparse']):
            key = "node_sparse" + str(i)
            if key in val_history:
                val_history[key].append(reg_val)
            else:
                val_history[key] = [reg_val]


        scheduler.step(train_data['loss'])
        print("Learning rate:",opt.param_groups[0]['lr'])
        best_model(train_data['acc'], val_data['acc'], model=model)

        
    # add values to the run history
    runs.add_history(train_history, "train")
    runs.add_history(val_history, "val")
    
    # other properties
    hist = {'skips': skips_to_list(model)}
    runs.add_history(hist, "")
            
    # update random seed
    conf.random_seed += 1
    cf.seed_torch(conf.random_seed)